In [2]:
from typing import Union
import os
import requests
import earthaccess
import pandas as pd
import datetime as dt
from datetime import date
from dateutil import parser
import geopandas
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.geometry.polygon import orient
from sentinel_tiles import sentinel_tiles
from emit_tools.generate_CMR_daterange_string import generate_CMR_daterange_string
from emit_tools.search_EMIT_L2A_reflectance import search_EMIT_L2A_reflectance

In [3]:
geojson_filename = "11SPS.geojson"
start_date = "2024-05-01"
end_date = "2024-05-30"

In [4]:
polygon = sentinel_tiles.footprint("11SPS")
polygon.wkt

'POLYGON ((-115.92423373599996 33.43490918800006, -114.74363157999994 33.419046707000064, -114.76862412 32.429359161000036, -115.93615876899997 32.444635394000045, -115.92423373599996 33.43490918800006))'

In [6]:
search_EMIT_L2A_reflectance(start_date, end_date, polygon)

<Response [200]>
{'feed': {'updated': '2024-08-23T01:59:32.900Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json', 'title': 'ECHO granule metadata', 'entry': [{'time_start': '2024-05-21T23:27:16.000Z', 'cloud_cover': '30', 'updated': '2024-05-23T23:03:47.000Z', 'dataset_id': 'EMIT L2A Estimated Surface Reflectance and Uncertainty and Masks 60 m V001', 'data_center': 'LPCLOUD', 'title': 'EMIT_L2A_RFL_001_20240521T232716_2414216_002', 'coordinate_system': 'GEODETIC', 'day_night_flag': 'DAY', 'time_end': '2024-05-21T23:27:28.000Z', 'id': 'G3029286621-LPCLOUD', 'original_format': 'UMM_JSON', 'granule_size': '3579.7861518859863', 'browse_flag': True, 'polygons': [['33.5352974 -115.9862595 32.9145088 -116.7228775 32.3046684 -116.208931 32.925457 -115.4723129 33.5352974 -115.9862595']], 'collection_concept_id': 'C2408750690-LPCLOUD', 'online_access_flag': True, 'links': [{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download EMIT_L2A_RFL_001_20240521T232716_24

,date,product,filename,URL,cloud_percent,geometry
0,2024-08-01,L2A_RFL,EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,"MULTIPOLYGON (((-115.9862595 33.5352974, -116...."
0,2024-08-01,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232716_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,"MULTIPOLYGON (((-115.9862595 33.5352974, -116...."
0,2024-08-01,L2A_MASK,EMIT_L2A_MASK_001_20240521T232716_2414216_002.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,"MULTIPOLYGON (((-115.9862595 33.5352974, -116...."
1,2024-08-01,L2A_RFL,EMIT_L2A_RFL_001_20240521T232728_2414216_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,"MULTIPOLYGON (((-115.3462372 34.0547523, -116...."
1,2024-08-01,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232728_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,"MULTIPOLYGON (((-115.3462372 34.0547523, -116...."
1,2024-08-01,L2A_MASK,EMIT_L2A_MASK_001_20240521T232728_2414216_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,"MULTIPOLYGON (((-115.3462372 34.0547523, -116...."
2,2024-08-01,L2A_RFL,EMIT_L2A_RFL_001_20240521T232740_2414216_004.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,"MULTIPOLYGON (((-114.7049332 34.5700073, -115...."
2,2024-08-01,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232740_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,"MULTIPOLYGON (((-114.7049332 34.5700073, -115...."
2,2024-08-01,L2A_MASK,EMIT_L2A_MASK_001_20240521T232740_2414216_004.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,"MULTIPOLYGON (((-114.7049332 34.5700073, -115...."
